# Introduction

Read: http://aosabook.org/en/sqlalchemy.html

What you would basically do to get some data from Pythons DB-API:

    connection = dbapi.connect(user="user", pw="pw", host="host")
    cursor = connection.cursor()

    cursor.execute("select * from user_table where name=?", ("jack",))

    print "Columns in result:", [desc[0] for desc in cursor.description]

    for row in cursor.fetchall():
        print "Row:", row

    cursor.close()
    connection.close()

# How SQLAlchemy does it?

In [ ]:
from sqlalchemy import create_engine

# Syntax: dialect[+dialect]://username:password@host/dbname
engine = create_engine("postgresql://alchemist@alchemist_db/alchemist")

result = engine.execute("SELECT version();")

print(result.fetchall())

*Note*: everything is handled behind the scenes!

### The Engine, Connection and ResultProxy

In [ ]:
# we can explicitly maintain a connection:
connection = engine.connect()  # Engine references a Dialect
print(type(engine))
print(type(connection))

In [ ]:
result = connection.execute("SELECT current_date;")
print(type(result))
print(result.fetchone())

In [ ]:
connection.close()  # but then we also must close it...

*Note*: slides

### Table and Column concepts

In [ ]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table, Column, 
    Integer, String
)

metadata = MetaData()

# define a Table with some Columns
users = Table('users', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String)
)

In [ ]:
engine.echo = True  # I wan't to see the SQL statements
metadata.create_all(engine)

## SQL expressions
SQLAlchemy renders the expressions as strings...

In [ ]:
ins = users.insert()
print(str(ins))

So, insert() renders the SQL INSERT with named bind parameters. When we provide values, only they get rendered:

In [ ]:
ins = users.insert().values(name='Seven of Nine')
print(str(ins))

But where is `Seven of Nine`?

In [ ]:
print(str(ins.compile().params))

Let's add that to the DB:

In [ ]:
connection = engine.connect()
result = connection.execute(ins)
print(type(result))
print(result.inserted_primary_key)

INSERT can also be done like this:

In [ ]:
ins = users.insert()
connection.execute(ins, id=17, name='Ellen Ripley')

Or, we can insert many at once:

In [ ]:
connection.execute(ins, [
    {'id': 18, 'name': 'Cameron Phillips'},
    {'id': 19, 'name': 'Sarah Connor'},
])

### SELECT

In [ ]:
from sqlalchemy import select
sel = select([users])
result = connection.execute(sel)

We have the resultset (ResultProxy object), let's see what is in it

In [ ]:
for usr in result:
    print(usr)

We can access the fields in many ways:

In [ ]:
result = connection.execute(sel)
usr = result.fetchone()
print('Name: {}'.format(usr['name']))
print('Name: {}'.format(usr[1]))
print('Name: {}'.format(usr[users.c.name]))

result.close() should be called now, as there are more results in there...

In [ ]:
result.close()

Pick the fields to select

In [ ]:
sel = select([users.c.name])
result = connection.execute(sel)
for usr in result:
    print(usr)

### select() is really smart

In [ ]:
# repeating for reference:
#
# users = Table('users', metadata,
#    Column('id', Integer, primary_key=True),
#    Column('name', String)
# ) 

from sqlalchemy import ForeignKey

addresses = Table('addresses', metadata,
    Column('id', Integer, primary_key=True),
    Column('user_id', None, ForeignKey('users.id')),
    Column('email_address', String, nullable=False)
)

metadata.create_all(engine)

In [ ]:
connection.execute(users.delete()) # starting with fresh data

In [ ]:
connection.execute(users.insert(), [
    {'id': 1, 'name': 'Seven of Nine'},
    {'id': 2, 'name': 'Cameron Phillips'},
    {'id': 3, 'name': 'Sarah Connor'},
])

connection.execute(addresses.insert(), [
    {'user_id': 1, 'email_address': 'seven_of_nine@voyager.dq'},
    {'user_id': 1, 'email_address': 'seven_of_nine@unimatrix.01'},
    {'user_id': 2, 'email_address': 'cameron@sky.net'},
    {'user_id': 3, 'email_address': 'sarah@sky.net'},
])

In [ ]:
for row in connection.execute(select([users, addresses])):
    print(row)

Whoa, a Cartesian product :), we need a WHERE in there

In [ ]:
sel = select([users, addresses]).where(users.c.id == addresses.c.user_id)
for row in connection.execute(sel):
    print(row)

Did you get the `where()` thingie?!  What does `==` do?

Check more examples: http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#operators

In [ ]:
print(users.c.id == addresses.c.user_id)

`label()`

In [ ]:
sel = select([(users.c.name + ' with email ' + addresses.c.email_address).label('Full'), addresses.c.email_address]).where(users.c.id == addresses.c.user_id)
for row in connection.execute(sel):
    print(row)

There are `and_(), or_(), not_(), like()` and so on...

### `text()`

In [ ]:
from sqlalchemy.sql import text
sel = text("SELECT users.name || ' with email ' || addresses.email_address as Full " 
           "FROM users, addresses WHERE users.id = addresses.user_id "
           "AND addresses.email_address LIKE :email1")
for row in connection.execute(sel, email1='%.net').fetchall():
    print(row)

### join()

In [ ]:
print(users.join(addresses))

In [ ]:
print(':)')

In [ ]:
print(select([users.c.name]).select_from(users.join(addresses)))

In [ ]:
sel = select([users.c.name]).select_from(users.join(addresses)).limit(2).offset(2)
result = connection.execute(sel)
for row in result:
    print(row)

`order_by()`, `asc()`, `desc()`, ...